In [207]:
import pandas as pd
import sqldf
from tqdm import tqdm, trange
from fuzzywuzzy import fuzz

In [208]:
df=pd.read_excel('Buyers transparency new coding0822.xlsx',sheet_name='data')
df.iloc[:, -10:] = df.iloc[:, -10:].fillna(-1)
df.iloc[:,2:5]=df.iloc[:,2:5].fillna('')

In [209]:
def finalize():
    global df
    df['buyertracemx']=df[['buyer1trace','buyer2trace','buyer3trace']].max()
    df['buyerknowmx']=df[['buyer1know','buyer2know','buyer3know']].max()

In [210]:
with open('info.txt') as info:
    info=[x for x in info.read().split('#') if x!='']

info={x.split('\n')[0].strip():{y.strip().rsplit(' ',maxsplit=1)[0].strip().lower():int(y.strip().rsplit(' ',maxsplit=1)[1]) for y in x.split('\n')[1:] if len(y.rsplit(' ',maxsplit=1))>1} for x in info}
# info
print([len(info[x]) for x in info])

[150, 150, 150, 150]


# direct match

In [211]:
df['buyer1pol']=df['buyer1name'].map(info['pol'])
df['buyer1gov']=df['buyer1name'].map(info['gov'])
for i in '123':
    for j in ['trace','know']:
        df['buyer'+i+j]=df['buyer'+i+'name'].map(info[j])

# fuzz match
## `pol`

In [213]:
unmatchedPol=sorted([x for x in df[df.buyer1pol.isna()].buyer1name.unique() if type(x)==str and x!=''])
shres=75
for i in unmatchedPol:
    firstMatch=[x for x in info['pol'] if x[0].lower()==i[0].lower()]
    possible=[x for x in firstMatch if fuzz.partial_ratio(i,x)>shres]
    if len(possible)>=1:
        print(i,":",end='',sep='')
        print(*possible,sep=',')

adida:adidas
aeropostale:aéropostale
american global logistics:american eagle
armani ga:armani
asics:asics corporation
aÃ©ropostale:aéropostale
bershka bsk espana s.a:bershka
columbia:columbia sportswear co
costco:costco – kirkland signature
dick's:dick's sporting goods
george:george at asda
gildan:gildan activewear
gu:guess?,gucci
guess:guess?
hanesbrands:hanes
hansae:hanes
helly hansen:hanes
hermes otto international:hermès
jc penney:jcpenney
jordache:jordan
kohl:kohl's
levis:levi strauss & co
lidl:lidl uk
llbean:ll bean
macys:macy's
new york:new york & co.,new yorker
primark us:primark
primary foreign service:primark
sainsbury:sainsburys – tu clothing
target australia:target
tesco:tesco – f&f
victoria secret:victoria's secret
youngone corp.:youngor
ys:ysl


In [214]:
with open ('fuzz.txt') as fz:
    fz=fz.read().split('\n')
    fz={x.split(':')[0]:x.split(':')[1] for x in fz}

fz

{'adida': 'adidas',
 'aeropostale': 'aéropostale',
 'armani ga': 'armani',
 'asics': 'asics corporation',
 'aÃ©ropostale': 'aéropostale',
 'bershka bsk espana s.a': 'bershka',
 'columbia': 'columbia sportswear co',
 'costco': 'costco – kirkland signature',
 "dick's": "dick's sporting goods",
 'george': 'george at asda',
 'gildan': 'gildan activewear',
 'guess': 'guess?',
 'hanesbrands': 'hanes',
 'jc penney': 'jcpenney',
 'kohl': "kohl's",
 'levis': 'levi strauss & co',
 'lidl': 'lidl uk',
 'llbean': 'll bean',
 'macys': "macy's",
 'new york': 'new york & co.',
 'primark us': 'primark',
 'primary foreign service': 'primark',
 'sainsbury': 'sainsburys – tu clothing',
 'target australia': 'target',
 'tesco': 'tesco – f&f',
 'victoria secret': "victoria's secret"}

## `gov`:
similar to `pol`

## match

In [231]:
# df.to_csv('temp0824.csv',index=0)
df.loc[df.buyer1pol.isna(),'buyer1pol']=df.loc[df.buyer1pol.isna(),'buyer1name'].map({x:info['pol'][fz[x]] for x in fz})
df.loc[df.buyer1gov.isna(),'buyer1gov']=df.loc[df.buyer1gov.isna(),'buyer1name'].map({x:info['gov'][fz[x]] for x in fz if fz[x] in info['gov'].keys()})
df.loc[df.buyer1trace.isna(),'buyer1trace']=df.loc[df.buyer1trace.isna(),'buyer1name'].map({x:info['trace'][fz[x]] for x in fz if fz[x] in info['trace'].keys()})
df.loc[df.buyer2trace.isna(),'buyer2trace']=df.loc[df.buyer2trace.isna(),'buyer2name'].map({x:info['trace'][fz[x]] for x in fz if fz[x] in info['trace'].keys()})
df.loc[df.buyer3trace.isna(),'buyer3trace']=df.loc[df.buyer3trace.isna(),'buyer3name'].map({x:info['trace'][fz[x]] for x in fz if fz[x] in info['trace'].keys()})
df.loc[df.buyer1know.isna(),'buyer1know']=df.loc[df.buyer1know.isna(),'buyer1name'].map({x:info['know'][fz[x]] for x in fz if fz[x] in info['know'].keys()})
df.loc[df.buyer2know.isna(),'buyer2know']=df.loc[df.buyer2know.isna(),'buyer2name'].map({x:info['know'][fz[x]] for x in fz if fz[x] in info['know'].keys()})
df.loc[df.buyer3know.isna(),'buyer3know']=df.loc[df.buyer3know.isna(),'buyer3name'].map({x:info['know'][fz[x]] for x in fz if fz[x] in info['know'].keys()})

# buyer2, buyer3
## trace

In [221]:
def process(num,name,shres=75):
    unmatched=sorted([x for x in df[df['buyer'+num+name].isna()]['buyer'+num+'name'].unique() if type(x)==str and x!=''])
    for i in unmatched:
        firstMatch=[x for x in info[name] if x[0].lower()==i[0].lower()]
        possible=[x for x in firstMatch if fuzz.partial_ratio(i,x)>shres]
        if len(possible)>=1:
            print(i,":",end='',sep='')
            print(*possible,sep=',')

process('2','know')

asics:asics corportaion
chaps:champion
cohl's:chico's
dsg:dick's sporting goods,dillards
g-star international:g-star
gu:guess?,gucci
hansae:hanes
hansbrands:hanes
hansoll:hanes
jc penny:jcpenney
jones:jack & jones
kohls:kohl's
levi's:levi strauss & co
masimmo dutti:massimo dutti
new york & company:new york & co.,new yorker
newlook:new look
newyorker:new york & co.,new yorker
ovs hong kong sourcing ltd.:ovs
s oliver:s. oliver
stani:stadivarius
tj- maxx:tj maxx
tommy:tommy hilfiger
under amour:under armour
urban:urban outfitters
w.angle:wrangler


In [233]:
for i in ['trace','know']:
    for j in '23':
        with open(i+j+'.txt') as file:
            supp={x.split(':')[0]: x.split(':')[1] for x in file.read().split('\n') if x!=''}
            supp={x:info[i][supp[x]] for x in supp if supp[x] in info[i].keys()}
            varName='buyer'+j+i
            df.loc[df[varName].isna(),varName]=df.loc[df[varName].isna(),'buyer'+j+'name'].map(supp)  

# manual check

# finalize + export

In [ ]:
finalize()
df.to_csv('indexCode.csv',index=0)